<a href="https://colab.research.google.com/github/r-kovalch/acter-ner/blob/main/notebooks/acter-cvalue-conf2-25-roberta-large-en.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Reennon/acter-ner

Cloning into 'acter-ner'...
remote: Enumerating objects: 467, done.
remote: Counting objects: 100% (236/236), done.
remote: Compressing objects: 100% (186/186), done.
remote: Total 467 (delta 138), reused 113 (delta 49), pack-reused 231 (from 1)
Receiving objects: 100% (467/467), 3.45 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (292/292), done.


In [2]:
!git clone https://github.com/AylaRT/ACTER

Cloning into 'ACTER'...
remote: Enumerating objects: 5448, done.
remote: Counting objects: 100% (5448/5448), done.
remote: Compressing objects: 100% (3304/3304), done.
remote: Total 5448 (delta 2684), reused 4893 (delta 2132), pack-reused 0 (from 0)
Receiving objects: 100% (5448/5448), 11.90 MiB | 17.04 MiB/s, done.
Resolving deltas: 100% (2684/2684), done.


In [3]:
%cd /content/ACTER

/content/ACTER


In [4]:
%cd /content/acter-ner/term_extractor

/content/acter-ner/term_extractor


In [5]:
ls

combine_corpora-en-fr-nl.sh      preprocess_acter_gliner.py
combine_corpora-en-fr.sh         preprocess_acter.py
combine_corpora_gliner_multi.sh  test_full.tsv
combine_corpora_gliner.sh        train_full.tsv
combine_corpora.sh               train_model.py
configs/                         train_spacy_model.py
dataset_processor.py             Untitled.ipynb
output/


In [6]:
!bash combine_corpora.sh


Processing directory: /content/ACTER/en/corp/annotated/annotations/sequential_annotations/iob_annotations/without_named_entities
Processed /content/ACTER/en/corp/annotated/annotations/sequential_annotations/iob_annotations/without_named_entities/corp_en_02_seq_terms.tsv: 59 sentences
Processed /content/ACTER/en/corp/annotated/annotations/sequential_annotations/iob_annotations/without_named_entities/corp_en_05_seq_terms.tsv: 51 sentences
Processed /content/ACTER/en/corp/annotated/annotations/sequential_annotations/iob_annotations/without_named_entities/corp_en_12_seq_terms.tsv: 278 sentences
Processed /content/ACTER/en/corp/annotated/annotations/sequential_annotations/iob_annotations/without_named_entities/corp_en_04_seq_terms.tsv: 62 sentences
Processed /content/ACTER/en/corp/annotated/annotations/sequential_annotations/iob_annotations/without_named_entities/corp_en_01_seq_terms.tsv: 227 sentences
Processed /content/ACTER/en/corp/annotated/annotations/sequential_annotations/iob_annota

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!rm train_full.tsv && \
  cp /content/drive/MyDrive/ucu/ner/datasets/train_cvalue_conf2.25.tsv train_full.tsv

In [9]:
# Cell X – programmatic IOB‐TSV → DocBin conversion for multiple splits
import spacy
from spacy.tokens import DocBin, Span
from pathlib import Path

# 1) Initialize blank pipeline (no models loaded)
nlp = spacy.blank("en")

# 2) List of (input TSV, desired output .spacy) pairs
splits = {
    "train_full.tsv":   "train_full.spacy",
    "test_full.tsv": "test_full.spacy",
}

# 3) Prepare output directory
out_dir = Path("output")
out_dir.mkdir(exist_ok=True)

# 4) Conversion loop
for tsv_name, spacy_name in splits.items():
    tsv_path = Path(tsv_name)
    if not tsv_path.exists():
        print(f"⚠️  Skipping missing {tsv_name}")
        continue

    docbin = DocBin(store_user_data=True)
    doc_count = ent_count = 0

    # read token-per-line, blank lines separate sentences → we group N sentences into one Doc
    # Here we’ll group every sentence as its own Doc (n_sents=1)
    words, labels = [], []
    with tsv_path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if words:
                    doc = spacy.tokens.Doc(nlp.vocab, words=words)
                    ents = []
                    # scan IOB labels to recover Span(start, end)
                    start = None
                    for i, tag in enumerate(labels):
                        if tag.startswith("B"):
                            if start is not None:
                                ents.append(Span(doc, start, i, label="TERM"))
                            start = i
                        elif tag.startswith("I"):
                            # continuation
                            continue
                        else:  # "O" or other
                            if start is not None:
                                ents.append(Span(doc, start, i, label="TERM"))
                                start = None
                    # catch final
                    if start is not None:
                        ents.append(Span(doc, start, len(labels), label="TERM"))

                    doc.ents = ents
                    docbin.add(doc)
                    doc_count += 1
                    ent_count += len(ents)
                    words, labels = [], []
                continue

            # parse token and IOB label (label may be "O" or "B-TERM"/"I-TERM")
            parts = line.split("\t")
            if len(parts) != 2:
                continue
            tok, tag = parts
            words.append(tok)
            # normalize to plain "B"/"I"/"O"
            if tag.startswith("B"):
                labels.append("B")
            elif tag.startswith("I"):
                labels.append("I")
            else:
                labels.append("O")

        # flush last sentence if missing trailing blank line
        if words:
            doc = spacy.tokens.Doc(nlp.vocab, words=words)
            ents = []
            start = None
            for i, tag in enumerate(labels):
                if tag == "B":
                    if start is not None:
                        ents.append(Span(doc, start, i, label="TERM"))
                    start = i
                elif tag == "O" and start is not None:
                    ents.append(Span(doc, start, i, label="TERM"))
                    start = None
            if start is not None:
                ents.append(Span(doc, start, len(labels), label="TERM"))
            doc.ents = ents
            docbin.add(doc)
            doc_count += 1
            ent_count += len(ents)

    # write out the DocBin
    out_path = out_dir / spacy_name
    docbin.to_disk(out_path)
    print(f"✅ Converted {tsv_name} → {out_path} "
          f"({doc_count} docs, {ent_count} entities)")


✅ Converted train_full.tsv → output/train_full.spacy (7113 docs, 9031 entities)
✅ Converted test_full.tsv → output/test_full.spacy (2833 docs, 5523 entities)


In [10]:
!mkdir input_data && \
  mv output/train_full.spacy input_data/train_full.spacy && \
  mv output/test_full.spacy input_data/test_full.spacy && \
  mkdir -p /content/drive/MyDrive/ucu/ner/output/acter-cvalue-conf2-25-roberta-large-en

In [11]:
!pip install 'spacy[transformers]' -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.2/756.2 kB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 128.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [12]:
!spacy train configs/config_base.cfg  --gpu-id 0 --output /content/drive/MyDrive/ucu/ner/output/acter-cvalue-conf2-25-roberta-large-en

ℹ Saving to output directory:
/content/drive/MyDrive/ucu/ner/output/acter-cvalue-conf2-25-roberta-large-en
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 177kB/s]
config.json: 100% 482/482 [00:00<00:00, 3.59MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 3.78MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 25.9MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 2.11MB/s]
2025-05-19 21:58:51.780410: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 21:58:51.798500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747691931.81

In [13]:
from google.colab import runtime
runtime.unassign()